In [3]:
import pandas as pd
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri


# Load seqICP library
robjects.r('''
    library(seqICP)
''')

pandas2ri.activate()

In [8]:
target = "XLE"

data_train = pd.read_clipboard()
date = data_train['date']
data_train = data_train.drop('date', axis=1)

data_train.tail()

,XLE,RPI,IPNMAT,RETAILx,USTPU,UEMPLT5,UEMP15T26,AMDMUOx,AWHMAN,PERMITNE,...,NONREVSL,M2REAL,EXJPUSx,S&P: indust,GS5,BAA,TB3SMFFM,T5YFFM,OILPRICEx,PCEPI
143,0.022315,0.004002,0.010035,0.010305,0.003854,-0.079234,-0.001021,0.011750,41.8,4.343805,...,0.001010,0.004834,-0.010761,0.041854,-0.05,-0.02,-0.05,0.76,0.016800,0.003419
144,0.051237,0.006101,-0.001811,0.011919,0.001030,0.053514,0.046915,0.015621,41.8,4.564348,...,-0.004051,0.003225,0.019379,0.039139,-0.01,-0.09,-0.01,0.73,0.019853,0.002233
145,-0.042699,0.004279,0.002938,-0.003534,0.000514,-0.031510,0.000000,0.003329,41.7,4.499810,...,0.000846,0.003810,-0.014825,-0.002524,-0.13,-0.04,-0.06,0.75,-0.027302,0.001313
146,0.000983,0.004279,0.002938,-0.003534,0.000514,-0.031510,0.000000,0.003329,41.7,4.499810,...,0.000846,0.003810,-0.014825,-0.002524,-0.13,-0.04,-0.06,0.75,-0.027302,0.001313
147,-0.127186,0.000382,-0.011762,-0.002176,0.001936,-0.028607,-0.137959,0.001310,41.6,4.369448,...,-0.000732,0.006494,-0.019707,-0.030981,-0.13,-0.12,-0.07,0.60,-0.087214,-0.001122


In [10]:
# convert dataframe to R objects
X_train = data_train.drop(target, axis=1).values
y_train = data_train[target]

X_train_r = numpy2ri.numpy2ri(X_train)
y_train_r = numpy2ri.numpy2ri(y_train)

# pass inputs to global variables
robjects.globalenv['Xmatrix'] = X_train_r
robjects.globalenv['Y'] = y_train_r

In [12]:
robjects.r(f'''
    seqICP_result <- seqICP(X = Xmatrix, Y = Y, 
        par.test = list(grid = seq(0, length(Y), length(Y)/10), complements = FALSE, link = sum,
        alpha = 0.05, B = 100), max.parents = {int(X_train_r.shape[1] / 2)}, stopIfEmpty = FALSE, silent = TRUE)
    seqICP_summary <- summary(seqICP_result)
    parent_set <- seqICP_result$parent.set

''')

# retrieve results from seqICP
parent_set = robjects.r['parent_set']

AttributeError: 'rpy2.rinterface.FloatSexpVector' object has no attribute 'shape'